In [1]:
import pandas as pd
import numpy as np
import csv
import re

#Deberia revisar lo de Counter que lo uso bastante y no conoxco muy bien la libreria
from collections import Counter


In [2]:
def texto_csv(df, column):
    
    #name of the document: as column_name_column.csv
    path = f'column_{column}.csv'
    
    """ Hay que quitar los vacios primero - lo he sacado de la funcion
    por no tranformar la tabla aqui """
   
    #2. We write every colum as a new line in the .csv
    non_empty_values = [str(i) for i in df[column]]
    
    with open(path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows([[value] for value in non_empty_values]) 
        
    print(f'Column successfully exported to {path}')

In [25]:
def format_price(x):
    
    """
    Decompose and fromate values for the colum price
    
    input: string 
    output: lsit, values of each key already casted:
        price: pos[0]: float
        descuento: pos[1]: float/np.nan(float) (where there is a discount or not)
    
    """
    
    # 1. Replace the € simbol and splits by the "/" ch -- generates a list of 2 elements (nums - text)
    x = x.replace("€", "").replace(",", ".").split("\n")
    
    # 4. Extracts and formats price value:
    price = float(x[0])
    
    # 5. Extracts and formats discount_price if not dicount np.nan
    discount = [np.nan if len(x) == 1 else float(x[1])][0]
    
    return [price, discount]

In [26]:
pwd

'/Users/Moni/Ironhack/my_ironhack/projects/final-project/notebooks/transformations'

In [27]:
df = pd.read_csv("../../data/scrape-consum_2024-03-11.csv")
df

,categoria,subcategoria,descripcion,unidades,precio,marca
0,Despensa,Aperitivos y frutos secos,Aceitunas Negras sin Hueso Lata 150 Gr,"7,93 € / 1 Kg","1,29 €\n1,19 €",CONSUM
1,Despensa,Aperitivos y frutos secos,Cacahuetes con Cáscara Natural Tostado Bolsa 2...,"4,00 € / 1 Kg","1,05 €\n1,00 €",CONSUM
2,Despensa,Aperitivos y frutos secos,Cacahuetes con Cáscara Tostados con Sal Bolsa ...,"4,00 € / 1 Kg","1,05 €\n1,00 €",CONSUM
3,Despensa,Aperitivos y frutos secos,Ganchito Maíz Sabor Queso Bolsa 80 Gr,"8,75 € / 1 Kg","0,75 €\n0,70 €",CONSUM
4,Despensa,Aperitivos y frutos secos,Aceitunas Rellenas Anchoa Suave Pack 3 3 x 50 Gr,"17,66 € / 1 Kg","2,99 €\n2,65 €",JOLCA
...,...,...,...,...,...,...
11294,Mascotas,Comida para gatos,Comida Húmeda Gatos Gourmet Perle Buey Sobr 4 ...,"9,82 € / 1 Kg","3,34 €",GOURMET
11295,Mascotas,Comida para gatos,"Comida Seca Gatos Ultima de Pollo 1,5 Kg","6,63 € / 1 Kg","9,95 €",ULTIMA
11296,Mascotas,Comida para gatos,Snacks para Gatos Felix Party Mix Original 60 Gr,"30,83 € / 1 Kg","1,85 €",FELIX
11297,Mascotas,Comida para gatos,Comida Seca Gatos Esterilizados Buey 800 Gr,"7,44 € / 1 Kg","5,95 €",ULTIMA


## Duplicados

In [28]:
def dupli_rows(df):
    "How many duplicate rows there is"
    duplicates = df.shape[0] - df.drop_duplicates().shape[0]
    print(f'There is {duplicates} duplicate rows')
    return 

dupli_rows(df)

There is 2312 duplicate rows


In [29]:
df.drop_duplicates(inplace=True)

In [30]:
# All collumns have values. 
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8987 entries, 0 to 11298
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   categoria     8987 non-null   object
 1   subcategoria  8987 non-null   object
 2   descripcion   8987 non-null   object
 3   unidades      8987 non-null   object
 4   precio        8987 non-null   object
 5   marca         8512 non-null   object
dtypes: object(6)
memory usage: 491.5+ KB


## PRECIO

### Precio

- No hay valores nulos

- El precio puede contener o 2 informaciones o 3:
    - El precio y a que hace referencia ese precio (pack)
    - El precio, precio si esta descontado y a que hace referencia ese precio(pack)

- Creamos una funcion que aplicamos a todos los elementos de la columna precio para formaterla. Aqui extraeremos los tres valores comentados antes. Cuando no haya descuento --> np.nan
    - price_info --> string
    - price_eu and price_disc_eu --> float
    
  **Note:** Interesante aqui el tema de apply(function) y luego el lambda. 
  
- Eliminamos la columna original

**NOTE:** En price_info, los valores son bastante consistentes siendo la mayoria de valoes (ud/pack). No obstante hay 14 valores que son de tipo ( xxx g) ej 500 g. Veremos a ver que hacemos. si los descartamos o como los tranformamos. No son una parte muy relevante del conjunto de datos ( 14 de 5035) pero tener en cuenta que hay que revisarlo: 


**Tranform Price column**

In [31]:
# Apply the function and get the element for each colum: 
df["price_eu"] = df["precio"].apply(format_price).apply(lambda x:x[0])
df["price_disc_eu"] = df["precio"].apply(format_price).apply(lambda x:x[1])

#Drop old column:
df.drop("precio", axis=1, inplace=True)

df.sample()

,categoria,subcategoria,descripcion,unidades,marca,price_eu,price_disc_eu
9923,Cuidado personal,Cuidado facial,Crema Facial Bakuchiol Antiarrugas. Ecológica ...,"10,98 € / 100 ml",FEEL FREE,5.49,NaN


In [32]:
df.rename(columns = {"unidades":"compare", "marca":"brand"}, inplace=True)
df.drop(columns = "price_disc_eu")
df = df[["categoria", "subcategoria", "descripcion", "price_eu", "compare", "brand"]]
df["brand"] = df["brand"].str.capitalize()

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_42934/4175834411.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["brand"] = df["brand"].str.capitalize()


In [46]:
df["supermercado"] = "CONSUM"
df_c.rename(columns= {"categoria":"category", "subcategoria":"subcategory", "descripcion":"description"}, inplace = True)

In [47]:
df_m = pd.read_csv("mercadona-2024.csv")

In [48]:
df_m["supermercado"] = "MERCADONA"

In [49]:
df = pd.concat([df_c, df_m])

In [50]:
df

,category,subcategory,description,price_eu,compare,brand,supermercado
0,Despensa,Aperitivos y frutos secos,Aceitunas Negras sin Hueso Lata 150 Gr,1.29,"7,93 € / 1 Kg",Consum,CONSUM
1,Despensa,Aperitivos y frutos secos,Cacahuetes con Cáscara Natural Tostado Bolsa 2...,1.05,"4,00 € / 1 Kg",Consum,CONSUM
2,Despensa,Aperitivos y frutos secos,Cacahuetes con Cáscara Tostados con Sal Bolsa ...,1.05,"4,00 € / 1 Kg",Consum,CONSUM
3,Despensa,Aperitivos y frutos secos,Ganchito Maíz Sabor Queso Bolsa 80 Gr,0.75,"8,75 € / 1 Kg",Consum,CONSUM
4,Despensa,Aperitivos y frutos secos,Aceitunas Rellenas Anchoa Suave Pack 3 3 x 50 Gr,2.99,"17,66 € / 1 Kg",Jolca,CONSUM
...,...,...,...,...,...,...,...
4384,Zumos,Tomate y otros sabores,Néctar de maracuyá Hacendado fruta de la pasió...,0.81,"0,81 €/L",Hacendado,MERCADONA
4385,Zumos,Tomate y otros sabores,Bebida de mango Hacendado sin azúcares añadido...,1.25,"1,25 €/L",Hacendado,MERCADONA
4386,Zumos,Tomate y otros sabores,Bebida de pomelo Hacendado sin azúcares añadid...,1.25,"1,25 €/L",Hacendado,MERCADONA
4387,Zumos,Tomate y otros sabores,Zumo de manzana selección Hacendado Brick 1 L,1.50,"1,50 €/L",Hacendado,MERCADONA


### EXPORT:

In [51]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

In [53]:
export_to_csv(df, "comparison-merca-consum")

DataFrame successfully exported to comparison-merca-consum.csv
